In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (11, 5)  #set default figure size
import numpy as np
from numpy import exp
from scipy.special import factorial
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import statsmodels.api as sm
from statsmodels.api import Poisson
from scipy import stats
from scipy.stats import norm
from scipy.stats import poisson
from statsmodels.iolib.summary2 import summary_col
from scipy.optimize import minimize
import mpmath
import math
from math import e
from math import gamma
import warnings
warnings.filterwarnings("ignore")

# Part I

<b>1. Consider the hard candy example from class. The associated data is in the file candy.csv. Develop the
following models discussed in class using maximum likelihood estimation (MLE):
<br>(a) the Poisson model,
<br>(b) the NBD model,
<br>(c) the Zero Inflated NBD model, and
<br>(d) Finite Mixture models for 2, 3, and 4 segments.
<br>Report your code and all relevant details, including the estimated values of the parameters for each model and
the corresponding log-likelihood values. Please add comments to your code to make it easy to understand.

In [6]:
df=pd.read_csv(r"C:\Users\cheta\Downloads\candy(1).csv")
df

,Packs,People
0,0,102
1,1,54
2,2,49
3,3,62
4,4,44
5,5,25
6,6,26
7,7,15
8,8,15
9,9,10


<b> (a) the Poisson model

In [8]:
params=0.1

In [9]:
inputs=df.copy()
inputs

,Packs,People
0,0,102
1,1,54
2,2,49
3,3,62
4,4,44
5,5,25
6,6,26
7,7,15
8,8,15
9,9,10


In [10]:
poisson_pmf = lambda k,l: k*np.log(l)-l*(np.log(e))-np.log(factorial(k))

In [11]:
def LL(params,inputs):
    lambda0=params
    e=inputs['Packs']
    p=inputs['People']
    sum=0
    for i in range(len(inputs)):
        sum+=poisson_pmf(e[i],lambda0)*p[i]
    return sum

In [12]:
def NLL(params, inputs):
    return(-(LL(params, inputs)))

In [13]:
final=minimize(NLL,
         args=inputs,
         x0=params,
         bounds=[(0.000001, None)],
        tol=1e-10,
        options={'ftol' : 1e-8},)

In [14]:
final

      fun: array([1544.99639045])
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([-2.27373677e-05])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 24
      nit: 10
     njev: 12
   status: 0
  success: True
        x: array([3.99122805])

In [16]:
print("The Value for Lambda is ",final.x[0],". The Value of Maximum Log likelihood is ",-1*final.fun[0] )

The Value for Lambda is  3.991228054895201 . The Value of Maximum Log likelihood is  -1544.996390448967


<b>(b) the NBD model,


In [17]:
params = np.array([0.1,0.1])

In [18]:
pmf_nbd=lambda a,n,k,t: (np.log(gamma(n+k))-(np.log(gamma(n))+np.log(factorial(k))))+(n*(np.log(a)-np.log(a+t)))+(k*(np.log(t)-np.log(a+t)))

In [19]:
def NBDLL(params,inputs):
    e=inputs['Packs']
    p=inputs['People']
    n=params[0]
    alpha=params[1]
    sum=0
    for i in range(len(inputs)):
        sum+=pmf_nbd(alpha,n,e[i],1)*p[i]
    return sum

In [20]:
def NLL(params, inputs):
    return(-(NBDLL(params, inputs)))

In [21]:
final=minimize(NLL,
         args=inputs,
         x0=params,
         bounds=[(0.000001, None),(0.000001, None)])

In [22]:
final

      fun: 1140.023746185597
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00020464, -0.00111413])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 39
      nit: 11
     njev: 13
   status: 0
  success: True
        x: array([0.99765785, 0.24996243])

In [23]:
print("The Value for n is ",final.x[0]," and Alpha is ",final.x[1],".The Value of Maximum Log likelihood is ",-1*final.fun )

The Value for n is  0.9976578501464919  and Alpha is  0.24996242988654846 .The Value of Maximum Log likelihood is  -1140.023746185597


<b>(c) the Zero Inflated NBD model

In [24]:
params = np.array([0.1,0.1,0.1])

In [25]:
pmf_zinbd=lambda a,n,k,t : ((gamma(n+k))/((gamma(n))*factorial(k)))*((a/(a+t))**n)*((t/(a+t))**k)

In [26]:
def ZINBDLL(params,inputs):
    e=inputs['Packs']
    p=inputs['People']
    pie=params[0]
    alpha=params[1]
    n=params[2]
    sumi=0
    ll = 0
    for i in range(len(inputs)):
        if i == 0:
            sumi = pie + ((1-pie) * (pmf_zinbd(alpha,n,e[i],1)))
        else:
            sumi = ((1-pie) * (pmf_zinbd(alpha,n,e[i],1)))
        ll += p[i] * np.log(sumi)

    return ll

In [27]:
def NLL(params, inputs):
    return(-(ZINBDLL(params, inputs)))

In [28]:
final=minimize(NLL,
         args=inputs,
         x0=params,
         bounds=[(0.000001, 0.999999),(0.000001, None),(0.000001, None)],
         tol=1e-10,
         options={'ftol' : 1e-8},)

In [29]:
final

      fun: 1136.1656408391289
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00500222,  0.00034106, -0.00045475])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 72
      nit: 16
     njev: 18
   status: 0
  success: True
        x: array([0.11310781, 0.334189  , 1.5039286 ])

In [30]:
print("The Value for pie is ",final.x[0]," and Alpha is ",final.x[1],". The Value of Maximum Log likelihood is ",-1*final.fun )

The Value for pie is  0.11310781428091622  and Alpha is  0.3341890033173927 . The Value of Maximum Log likelihood is  -1136.1656408391289


<b>(d) Finite Mixture models for 2 segments.


In [31]:
params = np.array([0.1,0.1,0.1])

In [32]:
poisson_pmf1 = lambda l1,k : (l1**k)*(e**(-l1))/factorial(k)

In [33]:
poisson_pmf2 = lambda l2,k : (l2**k)*(e**(-l2))/factorial(k)

In [34]:
def ZINBDLL(params,inputs):
    e=inputs['Packs']
    p=inputs['People']
    pie=params[0]
    l1=params[1]
    l2=params[2]
    sumi=0
    ll = 0
    for i in range(len(inputs)):
            sumi = (pie * (poisson_pmf1(l1,e[i]))) + ((1-pie) * (poisson_pmf2(l2,e[i])))
            ll += p[i] * np.log(sumi)

    return ll

In [35]:
def NLL(params, inputs):
    return(-(ZINBDLL(params, inputs)))

In [36]:
final=minimize(NLL,
         args=inputs,
         x0=params,
         bounds=[(0.000001, 0.999999),(0.000001, None),(0.000001, None)],
         tol=1e-10,
         options={'ftol' : 1e-8},)

In [37]:
final

      fun: 1188.8328271734968
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.31876731e-03, -1.81898941e-04,  9.09494627e-05])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 72
      nit: 16
     njev: 18
   status: 0
  success: True
        x: array([0.70088542, 1.80215233, 9.12069281])

In [38]:
print("The Value for pie is ",final.x[0]," and Lambda1 is ",final.x[1]," and Lambda2 is ",final.x[2],". The Value of Maximum Log likelihood is ",-1*final.fun )

The Value for pie is  0.7008854180927121  and Lambda1 is  1.802152325447988  and Lambda2 is  9.120692807782783 . The Value of Maximum Log likelihood is  -1188.8328271734968


<b>(d) Finite Mixture models for 3 segments.


In [80]:
params = np.array([1,1,1,2,1])

In [81]:
pie = lambda theta1, theta2: ((e**theta1)/(e**theta1 + e**theta2 + 1)) + ((e**theta2)/(e**theta1 + e**theta2 + 1)) + (1/(e**theta1 + e**theta2 + 1))

In [82]:
pie_sum = lambda theta1, theta2: (e**theta1 + e**theta2 + 1)

In [83]:
poisson_pmf = lambda l1,l2,l3,theta1,theta2,k : ((l1**k)*(e**(-l1))/factorial(k))*(e**theta1/pie_sum(theta1,theta2)) + ((l2**k)*(e**(-l2))/factorial(k))*(e**theta2/pie_sum(theta1,theta2)) + ((l3**k)*(e**(-l3))/factorial(k))*(1/pie_sum(theta1,theta2))

In [84]:
def SEG3LL(params,inputs):
    e=inputs['Packs']
    p=inputs['People']
    l1=params[0]
    l2=params[1]
    l3=params[2]
    theta1=params[3]
    theta2=params[4]
    sumi=0
    ll = 0
    for i in range(len(inputs)):
        sumi = poisson_pmf(l1,l2,l3,theta1,theta2,e[i])
        ll += p[i] * np.log(sumi)
    return ll

In [85]:
def NLL(params, inputs):
    return(-(SEG3LL(params, inputs)))

In [86]:
final=minimize(NLL,
         args=inputs,
         x0=params,
         bounds=[(0.000001, None),(0.000001, None),(0.000001, None),(None, None),(None, None)],
         tol=1e-10,
         options={'ftol' : 1e-8},)

In [87]:
final

      fun: 1132.0429842731146
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.00000000e+00, 4.54747354e-05, 1.97815098e-03, 7.04858394e-04,
       3.86535246e-04])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 156
      nit: 22
     njev: 26
   status: 0
  success: True
        x: array([11.21580038,  3.48331857,  0.2905524 , -0.43040618,  0.67443211])

In [88]:
print("The Value for lambda1 is ",final.x[0]," and Lambda2 is ",final.x[1]," and Lambda3 is ",final.x[2]," and theta1 is ",final.x[3]," and theta2 is ",final.x[4],". The Value of Maximum Log likelihood is ",-1*final.fun )

The Value for lambda1 is  11.21580038437127  and Lambda2 is  3.483318571593005  and Lambda3 is  0.29055239542745726  and theta1 is  -0.43040617532093817  and theta2 is  0.6744321103382401 . The Value of Maximum Log likelihood is  -1132.0429842731146


In [89]:
pie1= e**(final.x[3])/pie_sum(final.x[3],final.x[4])
pie2= e**(final.x[4])/pie_sum(final.x[3],final.x[4])
pie3= 1/pie_sum(final.x[3],final.x[4])

In [90]:
print("Value of pie1=",pie1)
print("Value of pie2=",pie2)
print("Value of pie3=",pie3)

Value of pie1= 0.17996557357392923
Value of pie2= 0.5432686018493437
Value of pie3= 0.2767658245767271


<b>(d) Finite Mixture models for 4 segments.


In [48]:
params = np.array([1,1,1,1,1,2,3])

In [49]:
pie = lambda theta1, theta2,theta3: ((e**theta1)/(e**theta1 + e**theta2 + e**theta3 + 1)) + ((e**theta2)/(e**theta1 + e**theta2 + e**theta3 + 1)) + ((e**theta3)/(e**theta1 + e**theta2 + e**theta3 + 1)) + (1/(e**theta1 + e**theta2 + e**theta3 + 1))

In [50]:
pie_sum = lambda theta1, theta2, theta3: (e**theta1 + e**theta2 + e**theta3 + 1)

In [51]:
poisson_pmf = lambda l1,l2,l3,l4,theta1,theta2,theta3,k : ((l1**k)*(e**(-l1))/factorial(k))*(e**theta1/pie_sum(theta1,theta2,theta3)) + ((l2**k)*(e**(-l2))/factorial(k))*(e**theta2/pie_sum(theta1,theta2,theta3)) + ((l3**k)*(e**(-l3))/factorial(k))*(e**theta3/pie_sum(theta1,theta2,theta3)) + ((l4**k)*(e**(-l4))/factorial(k))*(1/pie_sum(theta1,theta2,theta3))

In [52]:
def SEG3LL(params,inputs):
    e=inputs['Packs']
    p=inputs['People']
    l1=params[0]
    l2=params[1]
    l3=params[2]
    l4=params[3]
    theta1=params[4]
    theta2=params[5]
    theta3=params[6]
    sumi=0
    ll = 0
    for i in range(len(inputs)):
        sumi = poisson_pmf(l1,l2,l3,l4,theta1,theta2,theta3,e[i])
        ll += p[i] * np.log(sumi)
    return ll

In [53]:
def NLL(params, inputs):
    return(-(SEG3LL(params, inputs)))

In [54]:
final=minimize(NLL,
         args=inputs,
         x0=params,
         bounds=[(0.000001, None),(0.000001, None),(0.000001, None),(0.000001, None),(None, None),(None, None),(None, None)],
         tol=1e-10,
         options={'ftol' : 1e-8},)

In [55]:
final

      fun: 1130.0705916281213
 hess_inv: <7x7 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.0005457 , -0.00345608, -0.00127329,  0.00175078,  0.00277396,
       -0.00031832, -0.00147793])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 632
      nit: 62
     njev: 79
   status: 0
  success: True
        x: array([ 7.41912218,  0.20471897, 12.87263132,  3.00202207, -1.20013091,
       -0.72209099, -1.59835102])

In [56]:
print("The Value for lambda1 is ",final.x[0]," and Lambda2 is ",final.x[1]," and Lambda3 is ",final.x[2]," and Lambda4 is ",final.x[3]," and theta1 is ",final.x[4]," and theta2 is ",final.x[5],". The Value of Maximum Log likelihood is ",-1*final.fun )

The Value for lambda1 is  7.419122179159171  and Lambda2 is  0.20471896797905267  and Lambda3 is  12.87263131525968  and Lambda4 is  3.002022073340505  and theta1 is  -1.2001309104633455  and theta2 is  -0.7220909857650893 . The Value of Maximum Log likelihood is  -1130.0705916281213


In [73]:
pie1= e**(final.x[4])/pie_sum(final.x[4],final.x[5],final.x[6])

In [74]:
pie2= e**(final.x[5])/pie_sum(final.x[4],final.x[5],final.x[6])

In [75]:
pie3= e**(final.x[6])/pie_sum(final.x[4],final.x[5],final.x[6])

In [76]:
pie4=1/pie_sum(final.x[4],final.x[5],final.x[6])

In [78]:
print("Value of pie1=",pie1)
print("Value of pie2=",pie2)
print("Value of pie3=",pie3)
print("Value of pie4=",pie4)

Value of pie1= 0.1514010119410262
Value of pie2= 0.24419618755327793
Value of pie3= 0.10166792969361256
Value of pie4= 0.5027348708120832


<b>3. Based on the 2, 3, and 4-segment finite mixture models, how many packs are the following customers likely to
purchase over the next 8 weeks?
<br>(a) a customer who purchased 5 packs in the past week, and
<br>(b) a customer who purchased 9 packs in the past week.

In [93]:
#number of packs to be purchased over the next 8 weeks
# (Pi1* p(x=8|Lamba1))/( (Pi1* p(x=8|Lamba1))+ (Pi2* p(x=7|Lamba2))+ (Pi1* p(x=7|Lamba1)))

#Here we are using the 3 segment finite mixture model.
# Below is the code

params = np.array([1,1,1,2,1])

pie = lambda theta1, theta2: ((e**theta1)/(e**theta1 + e**theta2 + 1)) + ((e**theta2)/(e**theta1 + e**theta2 + 1)) + (1/(e**theta1 + e**theta2 + 1))

pie_sum = lambda theta1, theta2: (e**theta1 + e**theta2 + 1)

poisson_pmf = lambda l1,l2,l3,theta1,theta2,k : ((l1**k)*(e**(-l1))/factorial(k))*(e**theta1/pie_sum(theta1,theta2)) + ((l2**k)*(e**(-l2))/factorial(k))*(e**theta2/pie_sum(theta1,theta2)) + ((l3**k)*(e**(-l3))/factorial(k))*(1/pie_sum(theta1,theta2))

def SEG3LL(params,inputs):
    e=inputs['Packs']
    p=inputs['People']
    l1=params[0]
    l2=params[1]
    l3=params[2]
    theta1=params[3]
    theta2=params[4]
    sumi=0
    ll = 0
    for i in range(len(inputs)):
        sumi = poisson_pmf(l1,l2,l3,theta1,theta2,e[i])
        ll += p[i] * np.log(sumi)
    return ll


def NLL(params, inputs):
    return(-(SEG3LL(params, inputs)))


final=minimize(NLL,
         args=inputs,
         x0=params,
         bounds=[(0.000001, None),(0.000001, None),(0.000001, None),(None, None),(None, None)],
         tol=1e-10,
         options={'ftol' : 1e-8},)
final

pie1= e**(final.x[3])/pie_sum(final.x[3],final.x[4])
pie2= e**(final.x[4])/pie_sum(final.x[3],final.x[4])
pie3= 1/pie_sum(final.x[3],final.x[4])
lambda1=final.x[0]
lambda2=final.x[1]
lambda3=final.x[2]
print("Value of pie1=",pie1)
print("Value of pie2=",pie2)
print("Value of pie3=",pie3)
print("Value of lambda1=",lambda1)
print("Value of lambda2=",lambda2)
print("Value of lambda3=",lambda3)

Value of pie1= 0.17996557357392923
Value of pie2= 0.5432686018493437
Value of pie3= 0.2767658245767271
Value of lambda1= 11.21580038437127
Value of lambda2= 3.483318571593005
Value of lambda3= 0.29055239542745726


<b>A customer who purchased 5 packs in the past week.

In [94]:
poisson_pmf_one_var = lambda k,l: k*np.log(l)-l*(np.log(e))-np.log(factorial(k))

In [105]:
#calculating the denominator for customer who purchased 5 packs in segment 1.
Denominator=(poisson_pmf_one_var(5,lambda1)*pie1)+(poisson_pmf_one_var(5,lambda2)*pie2)+(poisson_pmf_one_var(5,lambda3)*pie3)

In [108]:
Numerator1=(poisson_pmf_one_var(5,lambda1)*pie1)
Numerator2=(poisson_pmf_one_var(5,lambda2)*pie2)
Numerator3=(poisson_pmf_one_var(5,lambda3)*pie3)

In [109]:
#p(s=1|x=5)
P_Seg_1=Numerator1/Denominator
#p(s=2|x=5)
P_Seg_2=Numerator2/Denominator
#p(s=3|x=5)
P_Seg_3=Numerator3/Denominator

In [110]:
#expected Purchases in one week based on the fact that customer has purchased 5 packs last week 
Exp_1week=lambda1*P_Seg_1+lambda2*P_Seg_2+lambda3*P_Seg_3

In [111]:
#Expected Purchases in 8 weeks
Exp_8weeks=Exp_1week*8

In [112]:
print("Expected purchases with 3 segment finite mixture model where customer purchased 5 packs last week is ", Exp_8weeks)

Expected purchases with 3 segment finite mixture model where customer purchased 5 packs last week is  20.55919703507159


<b> A customer who purchased 9 packs in the past week.

In [115]:
#calculating the denominator for customer who purchased 9 packs in segment 1.
Denominator=(poisson_pmf_one_var(9,lambda1)*pie1)+(poisson_pmf_one_var(9,lambda2)*pie2)+(poisson_pmf_one_var(9,lambda3)*pie3)

In [116]:
Numerator1=(poisson_pmf_one_var(9,lambda1)*pie1)
Numerator2=(poisson_pmf_one_var(9,lambda2)*pie2)
Numerator3=(poisson_pmf_one_var(9,lambda3)*pie3)

In [117]:
#p(s=1|x=9)
P_Seg_1=Numerator1/Denominator
#p(s=2|x=9)
P_Seg_2=Numerator2/Denominator
#p(s=3|x=9)
P_Seg_3=Numerator3/Denominator

In [118]:
#expected Purchases in one week based on the fact that customer has purchased 9 packs last week 
Exp_1week=lambda1*P_Seg_1+lambda2*P_Seg_2+lambda3*P_Seg_3

In [119]:
#Expected Purchases in 8 weeks
Exp_8weeks=Exp_1week*8

In [120]:
print("Expected purchases with 3 segment finite mixture model where customer purchased 9 packs last week is ", Exp_8weeks)

Expected purchases with 3 segment finite mixture model where customer purchased 9 packs last week is  13.050042128951457


# Part II

<b>1. Estimate all relevant parameters for Poisson regression using MLE. Report your code, the estimated parameters and the maximum value of the log-likelihood. What are the managerial takeaways — which customer
characteristics seem to be important?


In [104]:
kc=pd.read_csv(r'C:\Users\cheta\Downloads\articles (1).csv')
inputs=kc.copy()
inputs

,articles,female,married,kids,prestige,menpubs
0,0,0,1,0,2.52,7
1,0,1,0,0,2.05,6
2,0,1,0,0,3.75,6
3,0,0,1,1,1.18,3
4,0,1,0,0,3.75,26
...,...,...,...,...,...,...
910,11,0,1,2,2.86,7
911,12,0,1,1,4.29,35
912,12,0,1,1,1.86,5
913,16,0,1,0,1.74,21


In [105]:
# QUESTION 3 POISSON REGRESSION
poisson_pmf = lambda k,l: k*np.log(l)-l*(np.log(e))-np.log(factorial(k))


In [106]:
params = np.array([0.01,0.01,0.1,0.01,0.01,0.01])

In [107]:
params[1:len(params)]

array([0.01, 0.1 , 0.01, 0.01, 0.01])

In [108]:
def LL(params,inputs):
    k=inputs['articles'].tolist()
    kc1=inputs.drop(['articles'],axis=1)
    a=kc1.to_numpy().tolist()
    beta=params[1:len(params)]
    lambda0=params[0]
    lambdai=0
    sum2=0
    sum1=0
    
    for i in range(len(a)):
        sum1=0
        for j in range(len(a[i])):
            sum1=sum1+beta[j]*a[i][j]
        lambdai=lambda0*np.exp(sum1)
        sum2=sum2+poisson_pmf(k[i],lambdai)
    return sum2

In [109]:
def NegLL(params, inputs):
    return(-(LL(params, inputs)))

In [110]:
final=minimize(NegLL,
         args=inputs,
         x0=params,
         bounds=[(0.000001, None),(None, None), (None, None),(None, None), (None, None),(None, None)])

In [111]:
print("The Value for lambda is ",final.x[0],"Beta parameters are ",final.x[1:len(params)],".The Value of Maximum Log likelihood is ",-1*final.fun )

The Value for lambda is  1.3560303699561786 Beta parameters are  [-0.22451766  0.1553575  -0.18497029  0.01283066  0.02554278] .The Value of Maximum Log likelihood is  -1651.0563217267415


<b>2. Estimate all relevant parameters for NBD Regression using MLE. Report your code, the estimated parameters and the maximum value of the log-likelihood. What are the managerial takeaways — which customer
characteristics seem to be important?


In [112]:
#QUESTION 4 
#NBD REGRESSION 
kc=pd.read_csv(r'C:\Users\cheta\Downloads\articles (1).csv')
inputs=kc.copy()
inputs

,articles,female,married,kids,prestige,menpubs
0,0,0,1,0,2.52,7
1,0,1,0,0,2.05,6
2,0,1,0,0,3.75,6
3,0,0,1,1,1.18,3
4,0,1,0,0,3.75,26
...,...,...,...,...,...,...
910,11,0,1,2,2.86,7
911,12,0,1,1,4.29,35
912,12,0,1,1,1.86,5
913,16,0,1,0,1.74,21


In [113]:
params = np.array([0.1,0.1,0.1,0.1,0.1,0.1,0.1])

In [114]:
pmf_nbd=lambda a,n,k,t: (np.log(gamma(n+k))-(np.log(gamma(n))+np.log(factorial(k))))+(n*(np.log(a)-np.log(a+t)))+(k*(np.log(t)-np.log(a+t)))


In [115]:
def NBDLL(params,inputs):
    k=inputs['articles'].tolist()
    kc1=inputs.drop(['articles'],axis=1)
    a=kc1.to_numpy().tolist()
    beta=params[2:len(params)]
    alpha=params[0]
    n=params[1]
    lambdai=0
    sum2=0
    sum1=0
    for i in range(len(a)):
        sum1=0
        for j in range(len(a[i])):
            sum1=sum1+beta[j]*a[i][j]
        t=np.exp(sum1)
        sum2=sum2+pmf_nbd(alpha,n,k[i],t)
    return sum2

In [116]:
def NegLL(params, inputs):
    return(-(NBDLL(params, inputs)))

In [117]:
final=minimize(NegLL,
         args=inputs,
         x0=params,
         bounds=[(0.000001, None),(0.000001, None),(None, None), (None, None), (None, None), (None, None),(None, None)])


In [118]:
print("The Value for n is ",final.x[1]," and Alpha is ",final.x[0])
print("The values for the beta parameters are",final.x[2:len(params)])
print("The Value of Maximum Log likelihood is ",-1*final.fun)

The Value for n is  2.2642417083161877  and Alpha is  1.7524151977255495
The values for the beta parameters are [-0.21635544  0.15047396 -0.17641105  0.01523462  0.02908268]
The Value of Maximum Log likelihood is  -1560.9583396776768


<b>3. In this question, you will apply the ideas learned in this course to build a model that you have not seen before
– the Zero Inflated NBD Regression.
<br>First, recall that zero inflated models view 0s as coming from 2 sources - (i) from a fraction π who is 0 “by
type” (in the context of this problem, these are candidates who will never publish), and (ii) from the remaining
fraction (1 − π) who are likely to eventually become nonzero (these are candidates who will publish at some
point, but have not done so yet). You can assume that the candidates in the latter group are distributed as a
negative binomial (making the NBD regression appropriate for them).
<br>Explain the logic used in developing the model in detail. (hint: you do not need anything beyond
what you have learned in the class to do this.)
<br>Report your code, the estimated parameters and the maximum value of the log-likelihood. What are the
managerial takeaways — which customer characteristics seem to be important?


In [50]:
params=np.array((0.1,0.1,0.1,0.1,0.1,1,1,0.9))

In [51]:
pmf_nbd=lambda a,n,k,t: (np.log(gamma(n+k))-(np.log(gamma(n))+np.log(factorial(k))))+(n*(np.log(a)-np.log(a+t)))+(k*(np.log(t)-np.log(a+t)))

In [52]:
pmf_zinbd=lambda a,n,k,t : ((gamma(n+k))/((gamma(n))*factorial(k)))*((a/(a+t))**n)*((t/(a+t))**k)

In [53]:
df=pd.read_csv(r"C:\Users\cheta\Downloads\articles (1).csv")

In [54]:
inputs=df.copy()
df

,articles,female,married,kids,prestige,menpubs
0,0,0,1,0,2.52,7
1,0,1,0,0,2.05,6
2,0,1,0,0,3.75,6
3,0,0,1,1,1.18,3
4,0,1,0,0,3.75,26
...,...,...,...,...,...,...
910,11,0,1,2,2.86,7
911,12,0,1,1,4.29,35
912,12,0,1,1,1.86,5
913,16,0,1,0,1.74,21


In [66]:
def NBDLL(params,inputs):
    k=inputs['articles'].tolist()
    kc1=inputs.drop(['articles'],axis=1)
    a=kc1.to_numpy().tolist()
    beta=params[2:7]
    alpha=params[0]
    n=params[1]
    pie=params[2]
    sum2=0
    sum1=0
    ll=0
    for i in range(len(a)):
        sum1=0
        for j in range(len(a[i])):
            sum1=sum1+beta[j]*a[i][j]
        t=np.exp(sum1)
        if a[i] == 0:
            sum2 = sum2 + np.log(pie + ((1-pie) * (pmf_zinbd(alpha,n,k[i],t))))
        else:
            sum2 = sum2 + np.log((1-pie)) + (pmf_nbd(alpha,n,k[i],t))
    return sum2

In [67]:
def NegLL(params, inputs):
    return(-(NBDLL(params, inputs)))

In [68]:
NegLL(params,inputs)

3254.9312645281493

In [69]:
final=minimize(NegLL,
         args=inputs,
         x0=params,
         bounds=[(0.000001, None),(0.000001, None),(0.000001,0.999999),(None, None), (None, None), (None, None), (None, None),(None, None),],
         tol=1e-10,
         options={'ftol' : 1e-8})



In [119]:
print("The Value for n is ",final.x[1]," Alpha is ",final.x[0], " and Pie is ",final.x[2])
print("The values for the beta parameters are",final.x[2:len(params)])
print("The Value of Maximum Log likelihood is ",-1*final.fun)

The Value for n is  2.2642417083161877  Alpha is  1.7524151977255495  and Pie is  -0.21635544463093068
The values for the beta parameters are [-0.21635544  0.15047396 -0.17641105  0.01523462  0.02908268]
The Value of Maximum Log likelihood is  -1560.9583396776768
